<a href="https://colab.research.google.com/github/thor4/neuralnets/blob/master/projects/1-CNN/step_3_process_logits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Process logits
---
Now we can load logits generated from each model and extract confidence and accuracy from them

## 1: Load logits
These logits were created using the `step2-generation_predictions-mobilenetv2_model.ipynb` and `step2-generation_predictions-vanilla_model.ipynb` Jupyter notebooks. Run the cell to download a zip file from OSF then extract the excel files `logits_mobilenetv2.xlsx` and `logits_vanilla.xlsx` into the current directory

In [1]:
# @title Download logits

import requests, os
from zipfile import ZipFile

print("Start downloading and unzipping `Model logits`...")
name = 'logits'
fname = f"{name}.zip"
url = f"https://osf.io/k8fwn/download" #osf share link
r = requests.get(url, allow_redirects=True)
with open(fname, 'wb') as fh:
  fh.write(r.content) #download file

with ZipFile(fname, 'r') as zfile:
  zfile.extractall() #extract contents

if os.path.exists(fname):
  os.remove(fname) #delete zip file
else:
  print(f"The file {fname} does not exist")

print("Download completed.")

Start downloading and unzipping `Model logits`...
Download completed.


Import libraries first.

In [2]:
import pandas as pd
import numpy as np
from scipy.special import expit as sigmoid #get sigmoid function

Now we can load the logits into pandas dataframes

In [3]:
df_mnet = pd.read_excel("logits_mobilenetv2.xlsx") #mobilenetv2 logits
df_van = pd.read_excel("logits_vanilla.xlsx") #vanilla logits
print(df_mnet.head())
print(df_van.head())

     Logits  Labels
0  0.263464       0
1  0.380446       0
2 -0.105008       0
3  0.152609       0
4 -0.277837       0
     Logits  Labels
0 -2.320800       0
1 -0.269380       0
2 -2.161348       0
3 -0.795077       0
4 -2.513597       0


 -want to see how robust this is, take a range of confidence thresholds 
 -build out a function that takes n number of conditions/datasets and n number of logits and outputs the confidence and accuracy
After that, determine whether this model's logits produce the same dissociation results

-what about comparing the logits each model generates for each stimulus? some stimuli within datasets appear to drive confidence more than others. models do better with some stimulus images than others. maybe they specialize in a type of image

Define the confidence threshold for high confidence.

In [4]:
thres_min = -0.5
thres_max = 0.5

Next, let's define the function to get confidence and accuracy.

In [7]:
def get_conf_acc(df, thres_min, thres_max):
    df['Predictions'] = df['Logits'].apply(sigmoid) #sigmoid-transform the logits
    df['Predictions'] = np.where((df['Predictions'] < 0.5), 0, 1) #replace predictions with 0 or 1
    df['Accuracy'] = np.where((df['Labels'] == df['Predictions']), 1, 0)
    conditions = [
    (df['Logits'] < thres_min) | (df['Logits'] > thres_max), #high conf
    (df['Logits'] >= thres_min) | (df['Logits'] <= thres_max) #low conf
    ]
    values = ['high', 'low']
    df['Confidence'] = np.select(conditions, values)
    return df

Now we will assign confidence and accuracy to all 18,000 logits at once.

In [9]:
df = get_conf_acc(df_mnet,thres_min,thres_max)
print(df.iloc[5000:5010])

        Logits  Labels  Predictions Confidence  Accuracy
5000 -0.126332       0            0        low         1
5001 -1.941892       0            0       high         1
5002 -1.478281       0            0       high         1
5003 -1.361156       0            0       high         1
5004 -0.520181       0            0       high         1
5005  0.062038       0            1        low         0
5006 -0.320940       0            0        low         1
5007 -0.122390       0            0        low         1
5008 -1.939577       0            0       high         1
5009 -1.346852       0            0       high         1


In [22]:
df['Accuracy'].iloc[0:1000].mean() #avg accuracy for a dataset

0.543

In [23]:
df['Confidence'].iloc[0:1000].value_counts(normalize=True).loc['high'] #avg high conf for a dataset

0.04

In [17]:
df['Confidence'].value_counts(normalize=True).loc['high']

0.37422222222222223

In [ ]:
span = range(0, 18000, 1000)
df_results = pd.DataFrame(columns=['Logits','Labels']) #initialize dataframe
for n in span:
  set = df[n:n+1000] #extract dataset
  acc = set['Accuracy'].mean() #calculate avg accuracy
  conf = set['Confidence'].value_counts(normalize=True).loc['high'] #calculate avg high confidence
  df_results = pd.concat([df_results, pd.DataFrame({'Accuracy':acc,
                                                    'Confidence':conf})], 
                   axis=0, ignore_index=True) #append logits & labels to dataframe
  print(n,'to',n+1000)

Calculate confidence based on thresholds and add it to dataframe as a new column called `Confidence`

In [3]:
thres_min = -0.5
thres_max = 0.5

conditions = [
    (df_mnet['Logits'] < thres_min) | (df_mnet['Logits'] > thres_max), #high conf
    (df_mnet['Logits'] >= thres_min) | (df_mnet['Logits'] <= thres_max) #low conf
]

values = ['high', 'low']

df_mnet['Confidence'] = np.select(conditions, values)

print(df_mnet.iloc[5000:5010])

        Logits  Labels Confidence
5000 -0.126332       0        low
5001 -1.941892       0       high
5002 -1.478281       0       high
5003 -1.361156       0       high
5004 -0.520181       0       high
5005  0.062038       0        low
5006 -0.320940       0        low
5007 -0.122390       0        low
5008 -1.939577       0       high
5009 -1.346852       0       high


In [8]:
df_mnet['Logits'].to_numpy()

array([ 0.26346406,  0.38044614, -0.10500798, ...,  5.1039381 ,
        5.38165665,  5.02641058])

In [14]:
from scipy.special import expit as sigmoid #get sigmoid function
df_mnet['Predictions'] = df_mnet['Logits'].apply(sigmoid) #sigmoid-transform the logits



In [15]:
df_mnet.iloc[5000:5010]

,Logits,Labels,Confidence,Predictions
5000,-0.126332,0,low,0.468459
5001,-1.941892,0,high,0.125440
5002,-1.478281,0,high,0.185687
5003,-1.361156,0,high,0.204053
5004,-0.520181,0,high,0.372810
5005,0.062038,0,low,0.515505
5006,-0.320940,0,low,0.420447
5007,-0.122390,0,low,0.469441
5008,-1.939577,0,high,0.125694
5009,-1.346852,0,high,0.206385


In [16]:
df_mnet['Predictions'] = np.where((df_mnet['Predictions'] < 0.5), 0, 1)
df_mnet.iloc[5000:5010]

,Logits,Labels,Confidence,Predictions
5000,-0.126332,0,low,0
5001,-1.941892,0,high,0
5002,-1.478281,0,high,0
5003,-1.361156,0,high,0
5004,-0.520181,0,high,0
5005,0.062038,0,low,1
5006,-0.320940,0,low,0
5007,-0.122390,0,low,0
5008,-1.939577,0,high,0
5009,-1.346852,0,high,0


In [17]:
df_mnet['Accuracy'] = np.where((df_mnet['Labels'] == df_mnet['Predictions']), 1, 0)
df_mnet.iloc[5000:5010]

,Logits,Labels,Confidence,Predictions,Accuracy
5000,-0.126332,0,low,0,1
5001,-1.941892,0,high,0,1
5002,-1.478281,0,high,0,1
5003,-1.361156,0,high,0,1
5004,-0.520181,0,high,0,1
5005,0.062038,0,low,1,0
5006,-0.320940,0,low,0,1
5007,-0.122390,0,low,0,1
5008,-1.939577,0,high,0,1
5009,-1.346852,0,high,0,1


In [ ]:
predictions = tf.nn.sigmoid(predictions) #apply sigmoid activation function to transform logits to [0,1]
predictions = tf.where(predictions < 0.5, 0, 1) #round down or up accordingly since it's a binary classifier
accuracy = tf.where(tf.equal(predictions,label_batch),1,0) #correct is 1 and incorrect is 0

##NEXT: concatenate conf & acc into dataframe. cleanup code and get conf/acc for vanilla model. then split up by dataset

In [ ]:
df_mnet.shape

(18000, 2)

In [ ]:
df_mnet.iloc[999]

Logits   -0.072107
Labels    1.000000
Name: 999, dtype: float64

In [ ]:
test = df_mnet[0:1000] #first 1,000 entries
df_mnet[0:1000].shape

(1000, 2)

In [ ]:
test.iloc[-1] #last entry

Logits   -0.072107
Labels    1.000000
Name: 999, dtype: float64

last entry in `test` is same as the 999th entry of `df_mnet` since index starts at 0

In [ ]:
test.iloc[-1] == df_mnet.iloc[999] 

Logits    True
Labels    True
Name: 999, dtype: bool

the 17,999 entry in `df_mnet` is the last since index starts at 0

In [ ]:
df_mnet.iloc[17999] == df_mnet.iloc[-1]

Logits    True
Labels    True
Name: 17999, dtype: bool

In [ ]:
test2 = df_mnet[1000:2000]
df_mnet[1000:2000].shape

(1000, 2)

In [ ]:
test2.iloc[0]

Logits   -0.590789
Labels    0.000000
Name: 1000, dtype: float64

Let's establish our index ranges.

In [ ]:
span = range(0, 18000, 1000)
for n in span:
  print(n,'to',n+1000)

0 to 1000
1000 to 2000
2000 to 3000
3000 to 4000
4000 to 5000
5000 to 6000
6000 to 7000
7000 to 8000
8000 to 9000
9000 to 10000
10000 to 11000
11000 to 12000
12000 to 13000
13000 to 14000
14000 to 15000
15000 to 16000
16000 to 17000
17000 to 18000


In [ ]:
span = range(0, 18000, 1000)
for idx in span:
  print(idx,'to',idx+1000)
  df_mnet[idx:idx+1000]